In [19]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

#[len(names),len(prices), len(saleprices), len(soldouts),len(stores)]

In [57]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []

#Native Exotics
for page_number in range(1,1+1): #57pages
    URL = 'https://nativeexoticsonline.com/product-category/nepenthes/page/%d/' % page_number
    h = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
    req = Request(URL,headers=h) 
    page = urlopen(req).read() 

    #create soup single page
    soup = BeautifulSoup(page,'html.parser')
    
    name_box = soup.findAll('h2')
    name = name_box[1:]
        
    price_box = soup.findAll('price'})
    price = price_box

        try:
            price_box = container.find('span',attrs={'class':'product__price'})
            price = price_box.text.strip()
            separate_index = price.find('$') #look for 1st instance of $
            price = price[separate_index+1:] #remove the all characters before and including the string '$'
            prices = prices + [price]
            saleprices = saleprices + ['']           
        except AttributeError:
            #grab original price
            price_box = container.find('s')
            price = price_box.text.strip()
            separate_index = price.find('$') #look for 1st instance of $
            price = price[separate_index+1:] #remove the all characters before and including the string '$'
            prices = prices + [price]
            
            #grab new sales price
            price_box = container.find('span',attrs={'class':'product__price--on-sale'})
            price = price_box.text.strip()
            separate_index = price.find('$') #look for 1st instance of $
            price = price[separate_index+1:] #remove the all characters before and including the string '$'
            saleprices = saleprices + [price] 
        
        for i in range(12):
        try:
            soldout = container.find('strong',attrs={'class':'sold-out-text'}).text.strip()
            soldouts = soldouts + [soldout.lower()]
        except AttributeError:
            soldouts = soldouts + ['']
            
        names = names + [name]
        prices = prices + [price]
        stores = stores + ['Native Exotics']
        
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores}
df = pd.DataFrame(d)
df        

SyntaxError: invalid syntax (<ipython-input-57-de2627f18964>, line 20)

In [54]:
URL = 'https://nativeexoticsonline.com/product-category/nepenthes/page/%d/' % 57
#page = urlopen(URL)

h = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
req = Request(URL,headers=h) 
page = urlopen(req)
soup = BeautifulSoup(page,'html.parser')
a = soup.findAll('span',attrs={'class':'price'})
#container = containers[1]
a[0].text.strip()

'$\xa0199.00'

In [60]:
soldouts = []
for i in range(8):
    try:
        soldout = soup.find('span',attrs={'class':'soldout'}).text.strip()
        soldouts = soldouts + [soldout.lower()]
    except AttributeError:
        soldouts = soldouts + ['']
        
soldouts

['out of stock',
 'out of stock',
 'out of stock',
 'out of stock',
 'out of stock',
 'out of stock',
 'out of stock',
 'out of stock']

In [36]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []

#Carnivero
for page_number in range(1,1+15):
    URL = 'https://www.carnivero.com/collections/tropicals-pitcher-plants-nepenthes?page=%d' % page_number
    page = urlopen(URL)

    #create soup single page
    soup = BeautifulSoup(page,'html.parser')
    containers = soup.findAll('div',attrs={'class':'product grid__item medium-up--one-third small--one-half slide-up-animation animated'})
    
    for container in containers:
        name_box = container.find('div',attrs={'class':'product__title text-center'})
        name = name_box.text.strip()

        try:
            price_box = container.find('span',attrs={'class':'product__price'})
            price = price_box.text.strip()
            separate_index = price.find('$') #look for 1st instance of $
            price = price[separate_index+1:] #remove the all characters before and including the string '$'
            prices = prices + [price]
            saleprices = saleprices + ['']           
        except AttributeError:
            #grab original price
            price_box = container.find('s')
            price = price_box.text.strip()
            separate_index = price.find('$') #look for 1st instance of $
            price = price[separate_index+1:] #remove the all characters before and including the string '$'
            prices = prices + [price]
            
            #grab new sales price
            price_box = container.find('span',attrs={'class':'product__price--on-sale'})
            price = price_box.text.strip()
            separate_index = price.find('$') #look for 1st instance of $
            price = price[separate_index+1:] #remove the all characters before and including the string '$'
            saleprices = saleprices + [price] 
            
        try:
            soldout = container.find('strong',attrs={'class':'sold-out-text'}).text.strip()
            soldouts = soldouts + [soldout.lower()]
        except AttributeError:
            soldouts = soldouts + ['']
            
        names = names + [name]
        stores = stores + ['Carnivero']
        
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores}
df = pd.DataFrame(d)
df        

,Name,Price,Sale Price,Sold Out,Store
0,"Nepenthes spectabilis x ventricosa, BE-3373",19.99,,,Carnivero
1,Nepenthes Lady Luck,19.99,,,Carnivero
2,"Nepenthes bicalcarata, BE-3029",99.99,69.99,,Carnivero
3,Nepenthes ventricosa red - Carnivorous Pitcher...,23.95,,,Carnivero
4,"Nepenthes veitchii Bareo, BE-3734",69.99,,sold out,Carnivero
5,"Nepenthes vogelii, BE-3256",59.99,,,Carnivero
6,"Nepenthes ""Suki""",24.99,19.99,sold out,Carnivero
7,"Nepenthes alata x truncata, CAR-0003",19.99,,sold out,Carnivero
8,"Nepenthes ampullaria, BE-3681",44.99,39.99,,Carnivero
9,"Nepenthes robcantleyi, BE-3517",124.99,,sold out,Carnivero


In [73]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []

#Cook's Carnivorous Plants
for page_number in range(19,25):
    #select page number: nepenthes species highland(19) to nepenthes hybrid lowland(25)
    URL = 'http://www.flytraps.com/Scripts/prodList.asp?idCategory=%d' % page_number
    page = urlopen(URL)

    #create soup single page
    soup = BeautifulSoup(page,'html.parser')
    
    containers = soup.findAll('h3')
    containers_count = len(containers)

    #names of items
    for i in range(1,containers_count): #omitting the first container
        name_box = soup.findAll('h3')[i]
        name = name_box.text
        names = names + [name]
        stores = stores + ['Cook\'s Carnivorous Plants']
        soldouts = soldouts + ['']
    
    #prices of items
    for i in range(containers_count - 1):
        price_box = soup.findAll('del',attrs={'class':'CPprodLPriceV'})[i]
        price = price_box.text
        separate_index = price.find('$') #look for 1st instance of $
        price = price[separate_index+1:] #remove the all characters before and including the string '$'
        prices = prices + [price]
    
    #prices of items on sale
    containers_count_sale = len(soup.findAll('span',attrs={'class':'price'}))
    for i in range(containers_count_sale):
        price_box = soup.findAll('span',attrs={'class':'price'})[i]
        price = price_box.text
        separate_index = price.find('$') #look for 1st instance of $
        price = price[separate_index+1:] #remove the all characters before and including the string '$'
        price = price[:-3] #remove ' ea' from end of sales price
        saleprices = saleprices + [price]    
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores}
df = pd.DataFrame(d)
df

,Name,Price,Sale Price,Sold Out,Store
0,Nepenthes mira unpotted 4+ inch leafspan plant...,30.00,20.00,,Cook's Carnivorous Plants
1,Nepenthes mira unpotted 3 inch leafspan BE3876,20.00,15.00,,Cook's Carnivorous Plants
2,Nepenthes Borneo Exotics SP1 unpotted 4+ inch ...,26.00,18.00,,Cook's Carnivorous Plants
3,Nepenthes reinwardtiana green form unpotted 3 ...,26.00,14.00,,Cook's Carnivorous Plants
4,Nepenthes ventricosa madja as unpotted 3+ inch...,15.00,13.00,,Cook's Carnivorous Plants
5,Nepenthes hybrid veitchii x mira larger sized ...,95.00,60.00,,Cook's Carnivorous Plants
6,Nepenthes spathulata x spectabilis unpotted se...,16.00,14.00,,Cook's Carnivorous Plants
7,Nepenthes aristolochioides x burkei unpotted 4...,27.00,18.00,,Cook's Carnivorous Plants
8,Nepenthes aristolochioides x burkei unpotted 4...,25.00,18.00,,Cook's Carnivorous Plants
9,Nepenthes spectabilis x ventricosa unpotted 4+...,20.00,15.00,,Cook's Carnivorous Plants


In [77]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []

#California Carnivores
highland_neps = 'https://www.californiacarnivores.com/collections/highland-tropical-pitcher-plants'
lowland_neps = 'https://www.californiacarnivores.com/collections/lowland-nepenthes'
sections = [highland_neps,lowland_neps]

for page_number in range(2):
    #select page number
    URL = sections[page_number]
    page = urlopen(URL)

    #create soup and container object from single page
    soup = BeautifulSoup(page,'html.parser')
    containers = soup.findAll('div',attrs={'class':'grid__item large--one-quarter medium-down--one-half'})
    containers_sale = soup.findAll('div',attrs={'class':'grid__item large--one-quarter medium-down--one-half on-sale'})
    
    #grabs information of normal items
    for container in containers:
        name_box = container.find('p',attrs={'class':'grid-link__title'})
        name = name_box.text.strip()

        price_box = container.find('p',attrs={'class':'grid-link__meta'})
        price = price_box.text.strip()
        separate_index = price.find('$') #look for 1st instance of $
        price = price[separate_index+2:] #remove the all characters before and including the string '$ '
    
        names = names + [name]
        prices = prices + [price]
        saleprices = saleprices + ['']
        stores = stores + ['California Carnivores']
        soldouts = soldouts + ['']
        
    #grabs information of sale items
    for container in containers_sale:
        name_box = container.find('p',attrs={'class':'grid-link__title'})
        name = name_box.text.strip()

        price_box = container.find('p',attrs={'class':'grid-link__meta'})
        price = price_box.text.strip() #price of items on sale should be of form '$ xx.xx\n$ xx.xx'
        separate_index = price.rfind('$') #from end of string, look for 1st instance of $
        price_before = price[separate_index:] #gets original price
        price_before = price_before[2:] #remove the first 2 characters of the string '$ '
        price_after = price[:separate_index] #gets sale price
        price_after = price_after[2:] #remove the first 2 characters of the string '$ '
        price_after = price_after[:-1] #remove the last 2 characters of the string '$ '
    
        names = names + [name]
        prices = prices + [price_before]
        saleprices = saleprices + [price_after]
        stores = stores + ['California Carnivores']
        soldouts = soldouts + ['']
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores}
df = pd.DataFrame(d)
df

,Name,Price,Sale Price,Sold Out,Store
0,72 Hour Heat Pack,3.99,,,California Carnivores
1,Nepenthes ( x ventrata ) x alata (TC),14.95,,,California Carnivores
2,Nepenthes burbidgeae x veitchii (M-SG) Potted,58.95,,,California Carnivores
3,Nepenthes klossi (S-TC) Potted,298.00,,,California Carnivores
4,Nepenthes palawanensis (S-TC) Potted,98.95,,,California Carnivores
5,Nepenthes rajah x veitchii (M-TC) Potted,124.95,,,California Carnivores
6,Nepenthes robcantleyi x ( aristolochioides x s...,34.95,,,California Carnivores
7,Nepenthes robcantleyi x fusca (M-TC) Potted,38.95,,,California Carnivores
8,Nepenthes robcantleyi x veitchii (M-TC) Deluxe...,48.95,,,California Carnivores
9,Nepenthes singalana x ventricosa (M-TC) Deluxe...,18.95,,,California Carnivores


In [24]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []

#Pearl River Exotics
for page_number in range(1,1+4):
    #select page number
    URL = 'https://www.pearlriverexotics.com/collections/nepenthes?page=%d' % page_number
    page = urlopen(URL)

    #create soup and container object from single page
    soup = BeautifulSoup(page,'html.parser')
    containers = soup.findAll('a',attrs={'class':'grid-view-item__link'})

    for container in containers:
        name_box = container.find('div',attrs={'class':'h4 grid-view-item__title'})
        name = name_box.text.strip()

        price_box = container.find('span',attrs={'class':'product-price__price'})
        price = price_box.text.strip()
        price = price[1:] #remove the 1st character of the string '$'
    
        names = names + [name]
        prices = prices + [price]
        stores = stores + ['Pearl River Exotics']
        saleprices = saleprices + [''] #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!change when on sale
        
        #looks for sold out items
        try:
            soldout = container.find('span',attrs={'class':'product-price__sold-out'}).text.strip()
            soldouts = soldouts + [soldout.lower()]
        except AttributeError:
            soldouts = soldouts + ['']
            

d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores}
df1 = pd.DataFrame(d)
df1

,Name,Price,Sale Price,Sold Out,Store
0,72+ Hour Heat Pack - For shipping with plants,4.00,,,Pearl River Exotics
1,Cephalotus follicularis - Australian Pitcher P...,25.00,,sold out,Pearl River Exotics
2,Nepenthes burbidgeae BE-3834 (SEED-GROWN),59.00,,,Pearl River Exotics
3,Nepenthes burbidgeae x aristolochioides BE-3784,29.00,,,Pearl River Exotics
4,Nepenthes burkei x veitchii BE-3934,16.00,,,Pearl River Exotics
5,Nepenthes eymae BE-3736,30.00,,,Pearl River Exotics
6,Nepenthes glabrata x hamata - BE-3912,45.00,,,Pearl River Exotics
7,Nepenthes glandulifera BE-3766 *Seed-grown*,30.00,,,Pearl River Exotics
8,Nepenthes glandulifera BE-3879 (Single best cl...,30.00,,,Pearl River Exotics
9,Nepenthes hamata BE-3380 (G. Lumut) *SPECIMEN*,250.00,,sold out,Pearl River Exotics


In [87]:
df.append(df1)

,Name,Price,Sale Price,Sold Out,Store
0,72 Hour Heat Pack,3.99,,,California Carnivores
1,Nepenthes ( x ventrata ) x alata (TC),14.95,,,California Carnivores
2,Nepenthes burbidgeae x veitchii (M-SG) Potted,58.95,,,California Carnivores
3,Nepenthes klossi (S-TC) Potted,298.00,,,California Carnivores
4,Nepenthes palawanensis (S-TC) Potted,98.95,,,California Carnivores
5,Nepenthes rajah x veitchii (M-TC) Potted,124.95,,,California Carnivores
6,Nepenthes robcantleyi x ( aristolochioides x s...,34.95,,,California Carnivores
7,Nepenthes robcantleyi x fusca (M-TC) Potted,38.95,,,California Carnivores
8,Nepenthes robcantleyi x veitchii (M-TC) Deluxe...,48.95,,,California Carnivores
9,Nepenthes singalana x ventricosa (M-TC) Deluxe...,18.95,,,California Carnivores


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel ('database.xlsx')



df['cross3'] = np.full((df.shape[0],1),np.nan)
#df['cross4'] = np.full((df.shape[0],1),np.nan)

for n in range(df.shape[0]):
    name1 = df.loc[n,'cross1']
    name2 = df.loc[n,'cross2']
    
    if df.loc[n,'cross1'] == 'pure':
        df.loc[[n],['cross1']] = df.loc[n,'want'][3:] #replace 'pure' with name of species
        continue
    
    if ' x ' in name1: #if the cross has two species, put latter into cross3
        split_names = name1.split(' x ')
        df.loc[[n],['cross1']] = split_names[0]
        df.loc[[n],['cross3']] = split_names[1]    
    elif ' x ' in name2: #similar to above
        split_names = name2.split(' x ')
        df.loc[[n],['cross2']] = split_names[0]
        df.loc[[n],['cross3']] = split_names[1]
    
    #df.loc[[n],['s1']] = df.loc[n,'cross1']
    #df.loc[[n],['s2']] = df.loc[n,'cross2']

        
df

In [ ]:
d = dict()

for n in range(1):
    names = df.loc[[n],['cross1','cross2','cross3']]
    
    if True:
        continue
    
    
df.loc[[n],['cross1','cross2','cross3']]

In [ ]:
df.loc[4,['cross1','cross2']]

In [ ]:
subset = df[df['store']=='Bergen Water Gardens']
subset['price'].mean()

In [ ]:
table = pd.pivot_table(df, values='price', columns=['cross1'], aggfunc=np.mean)
table

In [ ]:
df1['e'] = Series(np.random.randn(sLength), index=df1.index) #appending column

In [ ]:
df.loc[[0],['cross1']] = 'hi'
df

In [ ]:
np.nan
#np.isnan(df.loc[0,'price'])
df.loc[0,'price'] = 'name'
df

In [ ]:
if ' x ' in str: 
    a = str.split(' x ')
    
a

In [ ]:
#Intro to Web Scraping and BeautifulSoup
#https://www.youtube.com/watch?v=XQgXKtPSzUI
